## Exploration des aides des l'ADEME.

Nous explorons des aides de l'ADEME, en open data :
https://data.ademe.fr/datasets/les-aides-financieres-de-l'ademe

Le but ici pour notre projet est de récupérer des couples aides/entreprises pour tenter d'automatiser cette association. Pour ramener ce matching dans le cadre de mission transition il faudrait pouvoir retrouver les aides de mission transition publiées, originaires de l'ADEME dans les donnée de l'ADEME. Ainsi nous obtiendrons des aides de mission transition enrichies des entreprises qui ont obtenue une subvention pour ces aides. Grâce à l'API SIRENE notament nous obtiendrons alors de nombreuses inforamtions pour tenter le matching.

### Récupération des aides de l'ADEME.

In [1]:
import os 
import pandas as pd
import requests
import numpy as np
from fuzzywuzzy import fuzz

In [2]:
# File path
DATA_FOLDER = "../data"
DATA_FILE = "Les aides financières ADEME-full.csv"
path_to_data = os.path.join(DATA_FOLDER, DATA_FILE)

# import into a Dataframe
df_ademe = pd.read_csv(path_to_data)
df_ademe


,Nom de l attribuant,idAttribuant,dateConvention,referenceDecision,nomBeneficiaire,idBeneficiaire,objet,montant,nature,conditionsVersement,...,notificationUE,pourcentageSubvention,location.lat,location.lon,DEPET,naf1etlib,naf2etlib,naf3etlib,naf4etlib,naf5etlib
0,ADEME,38529030900454,2021-04-12,21BFD0081,CLEMENT SAS,7.269205e+13,TREMPLIN pour la transition écologique des PME,6500.00,aide en numéraire,Echelonné,...,NaN,1.00,46.671862,4.365152,71,Industrie manufacturière,Fabrication de machines et équipements n.c.a.,Fabrication de machines de formage des métaux ...,Fabrication de machines de formage des métaux,Fabrication de machines-outils pour le travail...
1,ADEME,38529030900454,2021-03-25,21BFD0119,INDUSTRIE DOLOISE DE MICRO-MECANIQUE,3.950618e+13,TREMPLIN pour la transition écologique des PME,12150.00,aide en numéraire,Echelonné,...,NaN,1.00,47.105037,5.504615,39,Industrie manufacturière,"Fabrication de produits métalliques, à l'excep...",Traitement et revêtement des métaux ; usinage,Usinage,Mécanique industrielle
2,ADEME,38529030900454,2021-03-23,21BFD0123,TRANSDEV BFC SUD,3.391339e+13,TREMPLIN pour la transition écologique des PME,16000.00,aide en numéraire,Echelonné,...,NaN,1.00,46.790418,4.833945,71,Transports et entreposage,Transports terrestres et transport par conduites,Autres transports terrestres de voyageurs,Autres transports terrestres de voyageurs n.c.a.,Transports routiers réguliers de voyageurs
3,ADEME,38529030900454,2021-03-23,21BFD0137,LOCA TRAVAUX EURL,4.999836e+13,TREMPLIN pour la transition écologique des PME,16400.00,aide en numéraire,Echelonné,...,NaN,1.00,47.623341,6.957088,90,Activités de services administratifs et de sou...,Activités de location et location-bail,"Location et location-bail d'autres machines, é...",Location et location-bail de machines et équip...,Location et location-bail de machines et équip...
4,ADEME,38529030900454,2021-03-26,21BFD0142,RODESCHINI SAS,3.008465e+13,TREMPLIN pour la transition écologique des PME,21500.00,aide en numéraire,Echelonné,...,NaN,1.00,NaN,NaN,70,Construction,Travaux de construction spécialisés,Autres travaux de construction spécialisés,Autres travaux de construction spécialisés n.c.a.,Travaux de maçonnerie générale et gros œuvre d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12691,ADEME,38529030900454,2019-07-29,1905C0008,ASSOCIATION FRANCAISE DES PROFESSIONNELS DE L...,5.252716e+13,Soutien au programme d'actions 2019 de l'AFPG,88208.94,aide en numéraire,Echelonné,...,1.0,0.21,48.839829,2.346895,75,"Activités spécialisées, scientifiques et techn...",Activités des sièges sociaux ; conseil de gestion,Conseil de gestion,Conseil en relations publiques et communication,Conseil en relations publiques et communication
12692,ADEME,38529030900454,2019-07-12,1906C0002,MAITRES OUVRAGES PUBLICS,3.477621e+13,"Soutien à la réalisation du plan d'actions ""dé...",131930.69,aide en numéraire,Echelonné,...,1.0,0.20,45.771153,4.864377,69,"Activités spécialisées, scientifiques et techn...",Activités des sièges sociaux ; conseil de gestion,Conseil de gestion,Conseil pour les affaires et autres conseils d...,Conseil pour les affaires et autres conseils d...
12693,ADEME,38529030900454,2019-10-09,1916C0010,ASSOCIAT TECHNIQUE ENERGIE ENVIRONNEMENT,3.150628e+13,action 2019 : activités associatives portant s...,186724.54,aide en numéraire,Unique,...,1.0,0.27,48.888082,2.241744,92,"Activités spécialisées, scientifiques et techn...",Activités des sièges sociaux ; conseil de gestion,Conseil de gestion,Conseil pour les affaires et autres conseils d...,Conseil pour les affaires et autres conseils d...
12694,ADEME,38529030900454,2019-07-08,1966C0092,SOLE JULIEN,4.297205e+13,"""VVE - Vers une Ville Ecomobile"" / Conception ...",103570.00,aide en numéraire,Echelonné,...,1.0,0.36,43.301622,5.382030,13,"Arts, spectacles et activités récréatives","Activités créatives, artistiques et de spectacle","Activités créatives, artistiques et de spectacle",Cré

In [3]:
# Value counts for every columns 
for c in df_ademe.columns:
    print(df_ademe[c].value_counts())
    print("\n")

ADEME    12696
Name: Nom de l attribuant, dtype: int64


38529030900454    12696
Name: idAttribuant, dtype: int64


2020-11-26    150
2019-11-26    131
2019-11-28    130
2018-11-29    126
2020-11-24    104
             ... 
2019-12-12      1
2020-05-01      1
2019-02-01      1
2018-12-04      1
2019-11-30      1
Name: dateConvention, Length: 647, dtype: int64


1906C0018    10
1804C0025     9
2005C0015     8
1903C0003     8
2003C0001     7
             ..
2082C0008     1
2082C0009     1
2082C0010     1
2082C0011     1
20NOC0103     1
Name: referenceDecision, Length: 11795, dtype: int64


INSTITUT NATIONAL DE RECHERCHE POUR L'AGRICULTURE L'ALIMENTATION ET L'ENVIRONNEMENT    71
CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE                                           71
SUEZ RV REUNION                                                                        38
E. COMPAGNIE                                                                           35
SYNDICAT MIXTE POUR LA VALORISATION DES DECHE

On peut remarquer qu'il n'y a pas de numéro d'identification unique. La seule clé ici va être l'index. Attention alors si on récupère des données plus tard on ne pourat pas intersecter avec ces données. Il faudrat tout recommancer.

### Récupération des aides de Mission Transition.
Nous récupérons en particulier les offres de mission transition qui proviennent de l'ADEME.

In [4]:
def urlJsonToDataFrame(url):
    """
    This function return a pandas Dataframe using a given a json url.
    
            Parameter
            ---------
            url : sting
                An url which lead to a json
            
            Returns
            -------
            df : pandas DataFrame
                A pandas DataFrame of the json.
    """
    
    response = requests.get(url)
    response_json = response.json()
    df = pd.json_normalize(response_json)
    
    return df

In [5]:
df_mt_all = urlJsonToDataFrame("https://mission-transition-ecologique.beta.gouv.fr/api/temp/aids")
df_mt_all

,name,perimeter,goal,beneficiary,aidDetails,eligibility,conditions,fundingSourceUrl,applicationEndDate,applicationUrl,...,subventionRateUpperBound,subventionRateLowerBound,loanAmount,applicationStartDate,projectExamples,directAccess,types,id,funder.name,funder.id
0,Financer les études et tests préalables aux in...,NATIONAL,None,None,<p>\n <strong>\n Aide - « Études et tests pré...,<p>\n Critères d'éligibilité\n <br/>\n</p>\n<u...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,70.0,NaN,None,2021-11-09T15:42:42+00:00,<p>\n Actions éligibles :\n <br/>\n</p>\n<ul>\...,False,"[{'name': 'Aide financière', 'id': 1}]",189,ADEME,8
1,Soutenir financièrement le développement de l'...,NATIONAL,None,None,<p>\n Le dispositif de l'ADEME – ORPLAST (Obje...,"<p>\n <strong>\n Sont éligibles, les dossiers...",None,https://appelsaprojets.ademe.fr/aap/ORPLAST202...,2022-09-15T00:00:00+00:00,,...,NaN,NaN,None,2020-09-22T00:00:00+00:00,<p>\n En cas de questions relatives à l'appel ...,False,"[{'name': 'Aide financière', 'id': 1}]",178,ADEME,8
2,Financer les études de préfiguration d'une dém...,NATIONAL,None,None,<p>\n Aide - « Étude de préfiguration d'une dé...,,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,69.0,NaN,None,2021-11-09T15:42:48+00:00,<p>\n Actions éligibles :\n <br/>\n</p>\n<p>\n...,False,"[{'name': 'Aide financière', 'id': 1}]",203,ADEME,8
3,Réaliser des investissements de décarbonation ...,NATIONAL,None,None,<p>\n Plusieurs dispositifs sont ouverts aux e...,<p>\n <strong>\n Pour que la demande d'aide s...,None,https://www.asp-public.fr/aide-en-faveur-des-i...,2022-12-31T00:00:00+00:00,,...,NaN,NaN,None,2020-09-15T00:00:00+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",150,Ministère de la Transition Ecologique (MTE),52
4,Financer des audits énergétiques en entreprise...,NATIONAL,None,None,<p>\n <strong>\n Aide - « Financement des aud...,<p>\n Critères d'éligibilité : u\n ne étude de...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,70.0,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",161,ADEME,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,Soutenir l'émergence de solutions innovantes p...,REGIONAL,None,None,<p>\n <strong>\n La Région soutient l'émergen...,,None,https://www.iledefrance.fr/soutien-aux-actions...,None,,...,NaN,NaN,None,2021-10-05T19:09:55+00:00,<p>\n <strong>\n Les projets peuvent exploite...,False,"[{'name': 'Aide financière', 'id': 1}]",280,Conseil régional d'Ile de France,44
384,Sortir de vacance les logements locatifs privés,REGIONAL,None,None,<p>\n Le présent dispositif vise à remettre su...,<p>\n <strong>\n Bénéficiaires éligibles :\n ...,None,,None,,...,NaN,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",96,Conseil départemental de la Vendée,41
385,Soutenir l'économie de proximité en finançant ...,REGIONAL,None,None,<p>\n <strong>\n Aide - « Fonds régional des ...,,None,https://www.bourgognefranchecomte.fr/node/2835,2021-12-31T00:00:00+00:00,,...,NaN,NaN,None,2021-11-09T15:42:40+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",59,Conseil régional de Bourgogne-Franche-Comté,5
386,Soutenir le développement d'investissements ag...,REGIONAL,None,None,<p>\n La Région Occitanie est aux côtés des ac...,<p>\n <strong>\n Bénéficiaires\n </strong>\n ...,None,https://www.laregion.fr/PASS-Agritourisme,None,,...,NaN,NaN,None,2021-11-09T15:42:40+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",64,Conseil régional d'Occitanie,9


In [6]:
#Aids from ADEME into mission transition dataset
ademe_aids = df_mt_all["funder.name"] == "ADEME"
df_mt = df_mt_all[ademe_aids]
df_mt

,name,perimeter,goal,beneficiary,aidDetails,eligibility,conditions,fundingSourceUrl,applicationEndDate,applicationUrl,...,subventionRateUpperBound,subventionRateLowerBound,loanAmount,applicationStartDate,projectExamples,directAccess,types,id,funder.name,funder.id
0,Financer les études et tests préalables aux in...,NATIONAL,None,None,<p>\n <strong>\n Aide - « Études et tests pré...,<p>\n Critères d'éligibilité\n <br/>\n</p>\n<u...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,70.0,NaN,None,2021-11-09T15:42:42+00:00,<p>\n Actions éligibles :\n <br/>\n</p>\n<ul>\...,False,"[{'name': 'Aide financière', 'id': 1}]",189,ADEME,8
1,Soutenir financièrement le développement de l'...,NATIONAL,None,None,<p>\n Le dispositif de l'ADEME – ORPLAST (Obje...,"<p>\n <strong>\n Sont éligibles, les dossiers...",None,https://appelsaprojets.ademe.fr/aap/ORPLAST202...,2022-09-15T00:00:00+00:00,,...,NaN,NaN,None,2020-09-22T00:00:00+00:00,<p>\n En cas de questions relatives à l'appel ...,False,"[{'name': 'Aide financière', 'id': 1}]",178,ADEME,8
2,Financer les études de préfiguration d'une dém...,NATIONAL,None,None,<p>\n Aide - « Étude de préfiguration d'une dé...,,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,69.0,NaN,None,2021-11-09T15:42:48+00:00,<p>\n Actions éligibles :\n <br/>\n</p>\n<p>\n...,False,"[{'name': 'Aide financière', 'id': 1}]",203,ADEME,8
4,Financer des audits énergétiques en entreprise...,NATIONAL,None,None,<p>\n <strong>\n Aide - « Financement des aud...,<p>\n Critères d'éligibilité : u\n ne étude de...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,70.0,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",161,ADEME,8
5,Financer les équipements de lutte contre le ga...,NATIONAL,None,None,<p>\n <strong>\n Aide - « Équipements de lutt...,<p>\n Critères d'éligibilité :\n <br/>\n <ul>\...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,55.0,NaN,None,2021-11-09T15:42:42+00:00,<p>\n Vous pouvez prétendre à une aide pour le...,False,"[{'name': 'Aide financière', 'id': 1}]",190,ADEME,8
9,Gérer des biodéchets des acteurs économiques -...,NATIONAL,None,None,<p>\n <em>\n Mieux gérer vos biodéchets : l'A...,<p>\n Vous êtes une entreprise de restauration...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,NaN,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide en ingénierie', 'id': 2}]",39,ADEME,8
10,Investir pour la gestion des biodéchets des ac...,NATIONAL,None,None,"<p>\n <em>\n Pour mieux gérer les biodéchets,...",<p>\n Vous êtes une entreprise de restauration...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,NaN,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide financière', 'id': 1}, {'name'...",43,ADEME,8
11,Aider au déploiement du tri sélectif hors foyer,NATIONAL,None,None,<p>\n <strong>\n Aide - « Déploiement du tri ...,<p>\n <strong>\n Critères d'éligibilité :\n <...,None,https://agirpourlatransition.ademe.fr/entrepri...,2022-06-30T00:00:00+00:00,,...,50.0,NaN,None,2021-11-09T15:42:41+00:00,<p>\n Actions éligibles : les investissements ...,False,"[{'name': 'Aide financière', 'id': 1}]",177,ADEME,8
12,Subventionner des études préalables à la réali...,NATIONAL,None,None,<p>\n <em>\n Réalisation d'une étude territor...,<p>\n <strong>\n L'aide s'adresse aux acteurs...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,NaN,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",42,ADEME,8
13,Investir dans les centres de tri et de prépara...,NATIONAL,None,None,<p>\n <em>\n Vous souhaitez réaliser les inve...,<p>\n L'aide s'adresse aux acteurs publics ou ...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,NaN,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide financière', 'id': 1}, {'name'...",41,ADEME,8


Ici nous choisissons la clé : id. C'est domage que la colonne source_id qui était dans les premiers fichier n'apparais plus ici car elle correspond à l'id de aide territoire, dataset plus large si besoin.

### Lien entre les offres de l'ADEME et celles de Mission transition

La colonne objet des offres de l'ADEME et la colonne name des offres de Mission Transition sembles toutes les deux indiquer l'intitulé du projet. Cependant il n'existent pas de lien direct au sens ou l'intitulé n'est jamais exactement le même. Cependant, avec un examen plus minutieux on retrouve pratiquement les mêmes appellations. Voir les exemples ci dessous.

In [7]:
# inclusion
ademe_1 = df_ademe.objet.iloc[0] 
mt_1 = df_mt.name.iloc[21]  # id 44

print(ademe_1)
print(mt_1)

TREMPLIN pour la transition écologique des PME
Financer vos projets de transition écologique avec les aides financières du guichet « Tremplin pour la transition écologique des PME »


In [8]:
# Closeness
ademe_2 = df_ademe.objet.iloc[1582] 
mt_2= df_mt.name.iloc[29]  # id 44

print(ademe_2)
print(mt_2)

Réalisation d'une installation de production d'eau chaude solaire à la Maison de retraite des 5 étan
Financer les installations de production d'eau chaude solaire


Dans la première situation il suffit de chercher simplement la presence d'une chaîne de caractère dans l'autre. Dans le second cas nous nous proposons d'utiliser la distance de Levenshtein.

In [9]:
# Inclusion
ademe_1.lower() in mt_1.lower()

True

In [10]:
# Closeness
print(fuzz.ratio(ademe_2, mt_2))
print(fuzz.partial_ratio(ademe_2, mt_2))
print(fuzz.token_sort_ratio(ademe_2, mt_2))

66
87
64


In [11]:
# Just a first test !

In [12]:
aids_ademe = df_ademe.objet.values
aids_mt = df_mt.name.values

In [13]:
i = 0
Levenshtein_threshold = 60
for ademe_name in aids_ademe:
    for mt_name in aids_mt:
        
        ratio = fuzz.ratio(ademe_name,mt_name)
        
        
        if ratio > Levenshtein_threshold:
            i+=1
            print(ratio)
            print(ademe_name)
            print(mt_name)
            print('\n')


64
Mise en oeuvre du contrat patrimonial de développement des énergies renouvelables thermiques
Développer les énergies renouvelables thermiques


62
contrat de développement Patrimonial des énergies renouvelables thermiques de la SCIC BEL dans le Lo
Développer les énergies renouvelables thermiques


63
contrat de développement patrimonial des énergies renouvelables thermiques sur quatre installations
Développer les énergies renouvelables thermiques


61
réalisation d'une installation géothermique sur nappe assistée par une pompe à chaleur de 270 kW pou
Financer les installations géothermiques de production de chaleur et de froid


61
AAP STCo - Installation de production d'eau chaude solaire pour 10 villas (24 m²)
Aider à l'installation de production de chaleur biomasse/bois


68
AAP STCo - Installation de production d'eau chaude solaire pour 10 villas (24 m²)
Financer les installations de production d'eau chaude solaire


63
Aide à l'investissement pour la valorisation de déchets de 

63
Installation d'une PAC géothermique sur champ de sondes pour le Centre Aquatique d'Aigrefeuille sur
Tester l’intérêt de la géothermie sur champs de sondes pour le chauffage des bâtiments


61
installation PAC géothermique sur champ de sondes pour le Centre Aquatique de Chantonnay (85)
Tester l’intérêt de la géothermie sur champs de sondes pour le chauffage des bâtiments


69
Etude faisabilité PAC Géothermique sur champ de sondes pour le chauffage d'un bâtiment à Bécon les Granits (49)
Tester l’intérêt de la géothermie sur champs de sondes pour le chauffage des bâtiments


62
AAP Grandes installations - Solaire thermique - production d'eau chaude solaire (9 698 m²) pour le p
Financer les installations géothermiques de production de chaleur et de froid


61
AAP Grandes installations - Solaire thermique - production d'eau chaude solaire (9 698 m²) pour le p
Financer des installations de boucles d’eau tempérée géothermiques de production de chaleur et de froid


65
AAP Grandes installat

62
Réalisation de plusieurs études de faisabilité de réseaux de chaleur et de froid sur la Haute-Savoie
Financer une étude de faisabilité de récupération de chaleur fatale


72
Installation de méthanisation en cogénération
Financer des installations de méthanisation (injection, cogénération, chaleur)


61
Etude de faisabilité à l'implantation d'une déchetterie professionnelle
Financer des études préalables aux investissements dans les déchetteries professionnelles


62
Etude de faisabilité à l'implantetion d'une déchetterie professionnelle
Financer des études préalables aux investissements dans les déchetteries professionnelles


64
Financer l'étude de faisabilité pour le moulin de l'Abbaye de Fontenay (21)
Financer une étude de faisabilité pour une installation solaire thermique


61
Financer l'étude de faisabilité pour le moulin de l'Abbaye de Fontenay (21)
Financer des études de faisabilité chaufferie biomasse (Fonds chaleur)


62
Financer l'étude de faisabilité pour le moulin de l'

72
Etude de faisabilité pour une installation solaire thermique sur la piscine de Réquigny (56)
Financer une étude de faisabilité pour une installation solaire thermique


61
étude de faisabilité d'un réseau de chaleur bois énergie communal
Financer les études de faisabilité de chauffage des serres par énergie renouvelable


61
étude de faisabilité d'un réseau de chaleur bois énergie communal
Financer une étude de faisabilité de récupération de chaleur fatale


66
la réalisation d'une étude de faisabilité d'un réseau de chaleur alimenté par des énergies renouvela
Financer les études de faisabilité de chauffage des serres par énergie renouvelable


64
AAP Economie circulaire : Mise en place d'équipements de lutte contre le gaspillage alimentaire
Financer les équipements de lutte contre le gaspillage


61
mission d'assistance à maitrise d'ouvrage pour le projet du centre multimodal de Bastia
Financer une assistance à maîtrise d’ouvrage pour une chaufferie biomasse


67
Etude de faisabili

61
réalisation d'une assistance à maîtrise d'ouvrage du Conseil Régional Centre-Val de Loire, dans le c
Financer une assistance à maîtrise d’ouvrage pour une chaufferie biomasse


66
la réalisation d'une étude de récupération et de valorisation de chaleur fatale
Financer une étude de faisabilité de récupération de chaleur fatale


62
réalisation d'une étude de faisabilité pour la création d'un réseau de chaleur et d'un réseau de fro
Financer une étude de faisabilité de récupération de chaleur fatale


74
AAP ST - Etude de faisabilité pour l'installation de panneaux solaires thermiques
Financer une étude de faisabilité pour une installation solaire thermique


65
AAP ST - Etude de faisbilité pour une installation de production d'eau chaude solaire
Financer une étude de faisabilité pour une installation solaire thermique


71
AAP ST - Etude de faisbilité pour une installation de production d'eau chaude solaire
Financer les installations de production d'eau chaude solaire


67
Etude de fa

61
Etude de faisabilité pour une chaufferie bois et un réseau de chaleur à Arlanc (63)
Financer des études de faisabilité chaufferie biomasse (Fonds chaleur)


61
Etude de faisabilité récupération de chaleur sur le site d'INTERFORGE (ISSOIRE, 63)
Financer une étude de faisabilité de récupération de chaleur fatale


71
Réalisation d'une étude de faisabilité pour une installation de géothermie sur nappe pour la constru
Financer une étude de faisabilité pour une installation solaire thermique


61
Réalisation d'une étude de faisabilité pour la création d'un réseau de chaleur sur la commune de Mar
Financer une étude de faisabilité de récupération de chaleur fatale


62
Réalisation d'une étude de faisabilité pour la création d'une chaufferie biomasse pour l'école éléme
Financer des études de faisabilité chaufferie biomasse (Fonds chaleur)


68
Etude de faisabilité valorisation chaleur fatale (63)
Financer une étude de faisabilité de récupération de chaleur fatale


63
Etude de faisabilité c

61
Réalisation d'une étude de faisabilité géothermie
Financer une étude de faisabilité pour une installation solaire thermique


82
Réalisation d'une étude de faisabilité de récupération de chaleur fatale pour la SVEP
Financer une étude de faisabilité de récupération de chaleur fatale


61
Etude de faisabilité Hydrogéologique pour mise en place d'une installation géothermique - Phase 1
Financer une étude de faisabilité pour une installation solaire thermique


66
Etude de faisabilité d'un réseau de chaleur biomasse
Financer une étude de faisabilité de récupération de chaleur fatale


74
Réalisation d'une étude de faisabilité récupération de chaleur du groupe froid
Financer une étude de faisabilité de récupération de chaleur fatale


68
Réalisation d'une étude de faisabilité récupération de chaleur sur tours aéroréfrigérantes et compre
Financer une étude de faisabilité de récupération de chaleur fatale


65
Etude de faisabilité réseau de chaleur biomasse
Financer une étude de faisabilit

66
Etude de faisabilité de récupération de chaleur fatale sur le site de Caudebec Les Elbeuf (76)
Financer une étude de faisabilité de récupération de chaleur fatale


63
Cadre d'intervention EIT - Démarche d'écologie industrielle et territoriale sur le territoire de la
Financer les études de préfiguration d'une démarche d'écologie industrielle et territoriale


72
Développement de la démarche d'écologie industrielle et territorial
Financer les études de préfiguration d'une démarche d'écologie industrielle et territoriale


66
Accompagnement à la mise en place d'une démarche d'écologie Industrielle et Territoire sur la zone d
Financer les études de préfiguration d'une démarche d'écologie industrielle et territoriale


62
Accompagnement à la mise en place d'une démarche d'écologie Industrielle et Territoire sur la zone d
Accompagner les entreprises dans la mise en œuvre d'une démarche d'écoconception de leurs produits et services numériques


62
Création d'une conserverie locale solidai

In [14]:
i

420

### Création d'un dataset entre ADEME et Mission Transition
Dans cette partie nous allons créer un dataset en faisant matcher des aides de l'ADEME avec celle de Mission Transition. Pour ce faire nous allons dans un premier temps créer un dataset de scores, qui a chaque paire d'intitulé d'offre d'aide associe les scores de Levenshtein proposés par la bibliothèque fuzzywuzzy. Dans un second temps nous prosons une façon de matcher à partir des score de Leveinshtein. Enfin, nous créérons le dataset cobinant les données de l'ADEME et de Mission Transition.

In [15]:
# Data from ademe
df_ademe_ = pd.DataFrame(data = {"ademe_objet" : df_ademe["objet"], "ademe_index" : df_ademe.index })

# Data from mission transition
df_mt_ = pd.DataFrame(data = {"mt_name" : df_mt["name"], "mt_id" : df_mt["id"] })

# Product of ademe and mission transition
df_product = df_ademe_.merge(df_mt_, how='cross')

In [16]:
df_product

,ademe_objet,ademe_index,mt_name,mt_id
0,TREMPLIN pour la transition écologique des PME,0,Financer les études et tests préalables aux in...,189
1,TREMPLIN pour la transition écologique des PME,0,Soutenir financièrement le développement de l'...,178
2,TREMPLIN pour la transition écologique des PME,0,Financer les études de préfiguration d'une dém...,203
3,TREMPLIN pour la transition écologique des PME,0,Financer des audits énergétiques en entreprise...,161
4,TREMPLIN pour la transition écologique des PME,0,Financer les équipements de lutte contre le ga...,190
...,...,...,...,...
634795,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,Financer les études liées à la réduction des d...,184
634796,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,Accompagner tous les acteurs ligériens dans la...,9
634797,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,"Financer des études pour les collectivités, as...",174
634798,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,"Développer un projet de recherche, de développ...",140


In [52]:
def LevenshteinScores(n1, n2):
    """
    This function return a dictionary of differents levenshtein scores given by the fuzzywuzzy library.
    
            Parameter
            ---------
            n1 : sting
                String to compare with n2
                
            n2 : sting
                String to compare with n1
            
            Returns
            -------
            scores_dict : dictionary
                
    """
    n_1 = n1.lower()
    n_2 = n2.lower()
    
    l = fuzz.ratio(n_1, n_2)
    l_partial = fuzz.partial_ratio(n_1, n_2)
    l_token_sort = fuzz.token_sort_ratio(n_1, n_2)
    l_token_set = fuzz.token_set_ratio(n_1, n_2)
    
    scores_dict = {"L" : l, 
                   "L_partial" : l_partial, 
                   "L_token_sort" : l_token_sort, 
                   "L_token_set" : l_token_set}
    
    return score_dict


In [49]:
# Can takes few minutes to compute the Levenstein scores on the entire dataset.
df_levenshtein = df_product.apply(lambda x : LevenshteinScores(x["ademe_objet"], 
                                                               x["mt_name"]), 
                                  axis=1, result_type='expand')

In [50]:
df_scores = df_product.join(df_levenshtein)

In [51]:
df_scores

,ademe_objet,ademe_index,mt_name,mt_id,L,L_partial,L_token_sort,L_token_set
0,TREMPLIN pour la transition écologique des PME,0,Financer les études et tests préalables aux in...,189,32,48,33,35
1,TREMPLIN pour la transition écologique des PME,0,Soutenir financièrement le développement de l'...,178,35,41,37,37
2,TREMPLIN pour la transition écologique des PME,0,Financer les études de préfiguration d'une dém...,203,39,46,45,46
3,TREMPLIN pour la transition écologique des PME,0,Financer des audits énergétiques en entreprise...,161,38,43,39,39
4,TREMPLIN pour la transition écologique des PME,0,Financer les équipements de lutte contre le ga...,190,36,37,39,39
...,...,...,...,...,...,...,...,...
634795,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,Financer les études liées à la réduction des d...,184,37,42,41,40
634796,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,Accompagner tous les acteurs ligériens dans la...,9,38,42,46,46
634797,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,"Financer des études pour les collectivités, as...",174,41,51,43,45
634798,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,"Développer un projet de recherche, de développ...",140,39,54,41,49
